In [1]:
from torch_geometric.datasets import PCPNetDataset
from torch_geometric.transforms import ToSparseTensor, KNNGraph, Compose
from normal_diffusion.data.transforms import DistanceToEdgeWeight
import numpy as np

# Choose the root directory where you want to save the dataset
root = "../data/PCPNetDataset"

# Download and load the training dataset
dataset = PCPNetDataset(
    root=root,
    category="NoNoise",
    split="train",
    transform=Compose([KNNGraph(k=6), DistanceToEdgeWeight(), ToSparseTensor()]),
)

print(len(dataset))
first_collection = dataset[0]
first_collection.x = first_collection.x[:, :3]
print(first_collection.adj_t)
print(first_collection)

8
SparseTensor(row=tensor([    0,     0,     0,  ..., 99999, 99999, 99999]),
             col=tensor([52139, 56092, 58051,  ..., 51001, 82571, 99218]),
             val=tensor([0.1673, 0.1674, 0.1656,  ..., 0.1664, 0.1682, 0.1650]),
             size=(100000, 100000), nnz=600000, density=0.01%)
Data(x=[100000, 3], pos=[100000, 3], test_idx=[5000], adj_t=[100000, 100000, nnz=600000])


In [3]:
from normal_diffusion.models import GCNModel
model = GCNModel()
predicted_normals = model(graph_data=first_collection, t=1.0)
print(predicted_normals)


tensor([[ 4.0389, -2.4769, -2.7030],
        [ 2.6813, -1.4492, -1.5012],
        [ 3.7493, -2.2803, -2.5710],
        ...,
        [ 2.4581, -1.3523, -1.0705],
        [ 2.4606, -1.7351, -0.8122],
        [ 3.3111, -1.9130, -2.1515]], grad_fn=<AddBackward0>)
